# Import libraries

In [13]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import json
import pandas as pd
import csv

# Get Cache

# Concurrency

# Get Sessions

In [2]:
session = requests.Session()

# Get Urls

In [3]:
def get_page_links(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get(url)
    driver.find_element(By.XPATH,'//*[@id="uc-btn-accept-banner"]').click()

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    s_links = soup.select('li.search-results__item a')

    url_list = []
    for element in s_links:
        link = element["href"]
        if '/agency' in link or '/en/classified/' not in link: 
            pass

        elif 'https://www.immoweb.be' not in link:
            link = 'https://www.immoweb.be' + link
            url_list.append(link)
        else:
            url_list.append(link)
    return url_list


def get_element_url_list():
    all_urls = []
    for i,x in enumerate(range(1,15)):
        urls = get_page_links(f'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page={x}&orderBy=relevance')
        for url in urls:
            start = time.time()
            if url not in all_urls:
                all_urls.append(url)
            end = time.time()
        print('got Url',i,end-start)
    print(len(all_urls))
    return all_urls

#get houses
#get appartsments

In [4]:
url_list = get_element_url_list()

got Url 0 7.152557373046875e-07
got Url 1 1.9073486328125e-06
got Url 2 2.384185791015625e-06
got Url 3 3.0994415283203125e-06
got Url 4 2.86102294921875e-06
got Url 5 3.814697265625e-06
got Url 6 4.5299530029296875e-06
got Url 7 5.245208740234375e-06
got Url 8 5.9604644775390625e-06
got Url 9 6.67572021484375e-06
got Url 10 3.814697265625e-06
got Url 11 3.6716461181640625e-05
got Url 12 5.0067901611328125e-06
got Url 13 1.0728836059570312e-05
423


# Get Elements_properties_raw

In [5]:
def get_property(url):
    req_element = session.get(url)
    soup = BeautifulSoup(req_element.text, 'html.parser')
    element_info = soup.select('div.classified script')
    text = element_info[0].text

    return json.loads("".join(("".join(text.split('=')[1:]).split(";")[:-1])))

In [6]:
main_dictionary = dict()
start = time.time()
for index,url in enumerate(url_list):
    start = time.time()
    main_dictionary[url] = get_property(url)
    end = time.time()
    print('scraped',index,' time ',end-start,url)
end = time.time()
print(end-start)

scraped 0  time  0.5417935848236084 https://www.immoweb.be/en/classified/house/for-sale/woluwe-saint-lambert/1200/10060574?searchId=63387258266a7
scraped 1  time  0.21163582801818848 https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10145269?searchId=63387258266a7
scraped 2  time  0.19680142402648926 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gent-sint-denijs-westrem/9051/10145286?searchId=63387258266a7
scraped 3  time  0.15564870834350586 https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10141214?searchId=63387258266a7
scraped 4  time  0.2692103385925293 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/brussels-city/1000/9881253?searchId=63387258266a7
scraped 5  time  0.15837883949279785 https://www.immoweb.be/en/classified/apartment/for-sale/laeken/1020/10143541?searchId=63387258266a7
scraped 6  time  0.18099689483642578 https://www.immoweb.be/en/classified/apartme

In [7]:
for index,url in enumerate(url_list):
    print(index,url)

0 https://www.immoweb.be/en/classified/house/for-sale/woluwe-saint-lambert/1200/10060574?searchId=63387258266a7
1 https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10145269?searchId=63387258266a7
2 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gent-sint-denijs-westrem/9051/10145286?searchId=63387258266a7
3 https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10141214?searchId=63387258266a7
4 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/brussels-city/1000/9881253?searchId=63387258266a7
5 https://www.immoweb.be/en/classified/apartment/for-sale/laeken/1020/10143541?searchId=63387258266a7
6 https://www.immoweb.be/en/classified/apartment/for-sale/hoboken/2660/10143954?searchId=63387258266a7
7 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/zedelgem/8210/10074861?searchId=63387258266a7
8 https://www.immoweb.be/en/classified/new-real-estate-pr

# Save into json

In [8]:
with open("Main_datas.json","w") as main_data_file:
    main_data_file.write(json.dumps(main_dictionary))

# Get Elements_properties_list

In [9]:
data_main_list = list()
start = time.time()
for index,element in enumerate(main_dictionary.values()):
    element_data = {
    'id': element['id'],
    'url': list(main_dictionary.keys())[index],
    'Locality': element['property']['location']['postalCode'] if element['property']['location'] is not None else None,
    'Type of property': element['property']['type'] if element['property']['type'] is not None else None,
    'Subtype of property': element['property']['subtype'] if element['property']['subtype'] is not None else None,
    'Price': element['price']['mainValue'] if element['price'] is not None else None,
    'Type of sale': element['price']['type'] if element['price'] is not None else None,
    'Number of rooms': element['property']['roomCount'] if element['property']  is not None else None,
    'Area': element['property']['netHabitableSurface'] if element['property'] is not None else None,
    'Fully equipped kitchen': element['property']['kitchen']['type'] if element['property']['kitchen'] is not None else None,
    'Furnished': element['transaction']['sale']['isFurnished'] if element['transaction']['sale'] is not None else None,
    'Open fire': element['property']['fireplaceExists'] if element['property'] is not None else None,
    'Terrace': element['property']['hasTerrace'] if element['property'] is not None else None,
    'Terrace area': element['property']['terraceSurface'] if element['property'] is not None else None,
    'Garden': element['property']['hasGarden'] if element['property'] is not None else None,
    'Garden area': element['property']['gardenSurface'] if element['property'] is not None else None,
    'Surface of the land': element['property']['land']['surface'] if element['property']['land'] is not None else None,
    'Surface area of the plot of land': element['property']['land']['surface'] if element['property']['land'] is not None else None,
    'Number of facades ': element['property']['building']['facadeCount'] if element['property']['building'] is not None else 1,
    'Swimming pool': element['property']['hasSwimmingPool'] if element['property'] is not None else None,
    'State of the building': element['property']['building']['condition'] if element['property']['building'] is not None else None
    }
    data_main_list.append(element_data)
    print('Done',list(main_dictionary.keys())[index])
    
end = time.time()
print(end-start)

Done https://www.immoweb.be/en/classified/house/for-sale/woluwe-saint-lambert/1200/10060574?searchId=63387258266a7
Done https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10145269?searchId=63387258266a7
Done https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gent-sint-denijs-westrem/9051/10145286?searchId=63387258266a7
Done https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10141214?searchId=63387258266a7
Done https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/brussels-city/1000/9881253?searchId=63387258266a7
Done https://www.immoweb.be/en/classified/apartment/for-sale/laeken/1020/10143541?searchId=63387258266a7
Done https://www.immoweb.be/en/classified/apartment/for-sale/hoboken/2660/10143954?searchId=63387258266a7
Done https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/zedelgem/8210/10074861?searchId=63387258266a7
Done https://www.immoweb.be/en/cl

In [10]:
print (data_main_list)

[{'id': 10060574, 'url': 'https://www.immoweb.be/en/classified/house/for-sale/woluwe-saint-lambert/1200/10060574?searchId=63387258266a7', 'Locality': '1200', 'Type of property': 'HOUSE', 'Subtype of property': 'HOUSE', 'Price': 780000, 'Type of sale': 'residential_sale', 'Number of rooms': None, 'Area': 388, 'Fully equipped kitchen': 'NOT_INSTALLED', 'Furnished': False, 'Open fire': False, 'Terrace': True, 'Terrace area': 13, 'Garden': True, 'Garden area': 75, 'Surface of the land': 175, 'Surface area of the plot of land': 175, 'Number of facades ': 2, 'Swimming pool': False, 'State of the building': 'TO_BE_DONE_UP'}, {'id': 10145269, 'url': 'https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10145269?searchId=63387258266a7', 'Locality': '1200', 'Type of property': 'APARTMENT', 'Subtype of property': 'APARTMENT', 'Price': 275000, 'Type of sale': 'residential_sale', 'Number of rooms': None, 'Area': 83, 'Fully equipped kitchen': 'HYPER_EQUIPPED', 'Furnished

# Save information into CSV

In [15]:
def save_to_csv(list_dict):
    """a function that prints a csv starting from a dictionarires list 
        main(list_dict) """
    # we need at least 2 dict to create a csv file, so I made a df from 2 dict (the first 2 links that I have)
    df = pd.DataFrame([list_dict[0],list_dict[1]])
    df.to_csv("df_immo.csv", index=False)
    for i in  range(2, len(data_main_list)): #
        df = pd.read_csv("df_immo.csv")
        #I don't know if we are already cheacking for duplicates, but this will do that
        if data_main_list[i]["id"] not in df["id"]:
            #open csv in append mode ("a")
            file = open('df_immo.csv', "a")
            #use csv library to add a row to the csv file. The row has to be in a list format, so I've used the .values() method of the ditionary class
            csv.writer(file).writerow(data_main_list[i].values())           
    #at the end close the csv file
    file.close

It took 2,68 seconds for 400 houses, should take around 67 seconds for 10,000

In [16]:
start = time.time()
save_to_csv(data_main_list)
end = time.time()
print(end-start)

2.428807497024536
